In [1]:
import pandas as pd
import numpy as np
import json
import time
import datetime
import re
from pymongo import MongoClient
from bson.code import Code
import nltk
from difflib import SequenceMatcher
import random

client = MongoClient(host='localhost', port=27018)
db = client.kiva_DB 

folder = 'C:/Users/pablo/Desktop/DS - Primo anno Primo semestre/Data_Management/Progetto/{}.csv'
indicators = {'GDP' : pd.read_csv(folder.format('GDP')).set_index('Unnamed: 0'),
                 'GNI': pd.read_csv(folder.format('GNI')).set_index('Unnamed: 0'),
                 'GNI_atlas': pd.read_csv(folder.format('GNI_atlas')).set_index('Unnamed: 0'),
                 'GNI_pc': pd.read_csv(folder.format('GNI_pc')).set_index('Unnamed: 0')}

# 1) Quanti loans ci sono e a quanto ammontano in totale per ogni paese?

In [2]:
#Quanti loans ci sono e a quanto ammontano in totale per ogni paese?

def count_loans():

    pipeline = [
    {'$match':{}},
    {"$group": {"_id": "$location.country", "count": {"$sum": 1}}},
    ]
    
    total_loans_per_country_dict = {}
    res = list(db.loans.aggregate(pipeline))
    for doc in res:
        total_loans_per_country_dict[doc['_id']] = doc['count']
    
    return total_loans_per_country_dict

def avg_loans():

    pipeline = [
    {'$match':{}},
    {"$group": {"_id": "$location.country", "avg": {"$avg": "$funded_amount"}}},
    ]
    
    avg_loans_per_country_dict = {}
    res = list(db.loans.aggregate(pipeline))
    for doc in res:
        avg_loans_per_country_dict[doc['_id']] = doc['avg']
    
    return avg_loans_per_country_dict

def get_total_amount_loans():
    
    mapper = Code("""
              function (){emit(this.location.country, this.funded_amount)}
              """)

    reducer = Code("""
        function (key, values) {
        var total = 0;
        for (var i = 0; i < values.length; i++) {
            total += values[i];
                            }
                            return total;
                            }
        """)

    result = db.loans.map_reduce(mapper, reducer, "loans_totalAmount")

    total_amounts_per_country_dict = {}

    for doc in result.find():
        total_amounts_per_country_dict[doc['_id']] = doc['value']
    
    return total_amounts_per_country_dict
    
    

def get_time_period():
    
    pipeline = [
    {'$match':{}},
    {"$group": {"_id": "$location.country", "min": {"$min": "$funded_date"}, "max": {'$max':'$funded_date'}}},
    ]
    res = db.loans.aggregate(pipeline = pipeline)
    starts = {}
    ends = {}
    for d in res:
        starts[d['_id']] = d['min'][0:10]
        ends[d['_id']] = d['max'][0:10]
    return [starts, ends]


def simple_dict_to_frame(d,col1,col2):
    return pd.Series(d).to_frame().reset_index().rename(columns = {'index':col1, 0: col2})



In [4]:
t1 = time.time()
query1 = simple_dict_to_frame(count_loans(),'Country','Total loans')
print('Time elapsed to solve query: ', time.time() - t1)

query1['Total amount of loans'] = simple_dict_to_frame(get_total_amount_loans(),'Country','Total amount of loans')['Total amount of loans']
query1['Average loan amount'] = simple_dict_to_frame(avg_loans(),'Country','Average amount of loans')['Average amount of loans']
query1['Date of first loan registered'] = simple_dict_to_frame(get_time_period()[0],'Country','Date of first loan registered')['Date of first loan registered']
query1['Date of last loan registered'] = simple_dict_to_frame(get_time_period()[1],'Country','Date of last loan registered')['Date of last loan registered']

#query1.to_csv('C:/Users/pablo/Desktop/DS - Primo anno Primo semestre/Data_Management/Progetto/Output_queries/query1.csv')
query1.head()

OperationFailure: Could not find host matching read preference { mode: "primary" } for set kiva_s2

# 2) Quanti loans vengono chiesti ogni anno per un paese esempio (Perù)? 

In [ ]:
#Quanti loans vengono chiesti ogni giorno per un paese esempio? (Peru)
def extract_date(d):
    
    regex = re.match('(\d\d\d\d)-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d)',d['Date'])

    return datetime.datetime(year=int(regex.group(1)),month= int(regex.group(2)), day= int(regex.group(3)), hour = int(regex.group(4)), minute = int(regex.group(5)), second = int(regex.group(6)))


pipeline2 = [
    {'$match':{'location.country':'Peru',
     'funded_date':{'$exists':'true'}
              }
    },
    {"$group": {"_id": {'Date':"$funded_date"},
                "Number of loans":{"$sum": 1},
                "Amount funded (average)": {"$avg": "$funded_amount"}
               }
    }
    ]
query2 = pd.DataFrame()
t2 = time.time()
res2 = list(db.loans.aggregate(pipeline2))
print('Time elapsed to solve query: ', time.time() - t2)
if(len(res2)==0):
    print('Uncorrect query')
else:
    query2 = pd.DataFrame(res2)
    query2['Day (yy-mm-dd)'] = query2['_id'].apply(lambda d: str(extract_date(d))[:11])
    query2['Hour (hh:mm:ss)'] = query2['_id'].apply(lambda d: str(extract_date(d))[11:])
    query2 = query2[['Day (yy-mm-dd)','Hour (hh:mm:ss)','Amount funded (average)','Number of loans']].sort_values(by = 'Day (yy-mm-dd)').set_index(['Day (yy-mm-dd)','Hour (hh:mm:ss)'])
query2.to_csv('C:/Users/pablo/Desktop/DS - Primo anno Primo semestre/Data_Management/Progetto/Output_queries/query2.csv'

# 3) C'è qualche correlazione tra gli indici economici e il numero (e/o l'ammontare) dei loans finanziati in quel paese? 

In [154]:
#C'è qualche correlazione tra gli indici economici e il numero (e/o l'ammontare) dei loans finanziati in quel paese?

def get_year(d):
    return int(d['y'])

def indicator_to_number(d):
    r = re.match('(.*)\..*',str(d))
    if(r is None):
        return np.nan
    else:
        return int(r.group(1).replace(',',''))

pipeline3 = [
    {'$match': {
        'funded_amount':{'$exists':'true'},
        'funded_date':{'$exists':'true'},
        }
    },
    {'$project':{
        'year':{'$substr':["$funded_date",0,4]},
        'country':'$location.country',
        'amount': '$funded_amount',
        'date': "$funded_date",
        'GNI':'$location.GNI',
        'GNI_pc':'$location.GNI_pc',
        }
    },
    {'$group':{
        '_id':{'y':'$year','c':'$country'},
        'Number of loans funded':{'$sum':1},
        'Total loans amount': {'$sum':'$amount'},
        'Average loan amount': {'$avg':'$amount'},
        'Date of first loan of the year': {'$min':'$date'},
        'Date of last loan of the year': {'$max':'$date'},
        'GNI':{'$min':'$GNI'},
        'GNI per capita':{'$min':'$GNI_pc'},
                }
    }
]

t3 = time.time()
res3 = list(db.loans.aggregate(pipeline3))
print('Time elapsed to solve query: ', time.time() - t3)
query3 = pd.DataFrame(res3)

query3['Year'] = query3['_id'].apply(lambda d:int(d['y']))
query3['Country'] = query3['_id'].apply(lambda d:str(d['c']))
query3['Date of first loan of the year'] = query3['Date of first loan of the year'].apply(lambda d:d[0:10])
query3['Date of last loan of the year'] = query3['Date of last loan of the year'].apply(lambda d:d[0:10])
query3['GNI'] = query3['GNI'].apply(indicator_to_number)
query3['GNI per capita'] = query3['GNI per capita'].apply(indicator_to_number)
query3.drop('_id', axis=1, inplace=True)
query3 = query3.set_index(['Country','Year'],).sort_index()
query3.drop(labels =[2017,2018],level='Year', inplace=True)
#query3.to_csv('C:/Users/pablo/Desktop/DS - Primo anno Primo semestre/Data_Management/Progetto/Output_queries/query3.csv')
query3